In [1]:
# from TransformerBlock import TransformerBlock
import torch
import torch.nn as nn
import math
from enum import Enum

from LayerNorm import LayerNorm
from Block import Block
from CausalSelfAttention import CausalSelfAttention
from MLP import MLP

In [2]:
torch.cuda.is_available()

True

In [3]:
import inspect
import torch.nn.functional as F


class GPT(torch.nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.vocab_size is not None
        assert config.block_size is not None
        self.config = config

        self.transformer = nn.ModuleDict(
            dict(
                wte=nn.Embedding(config.vocab_size, config.n_embd),
                wpe=nn.Embedding(config.block_size, config.n_embd),
                drop=nn.Dropout(config.dropout),
                h=nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
                ln_f=LayerNorm(config.n_embd, bias=config.bias),
            )
        )
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        # with weight tying when using torch.compile() some warnings get generated:
        # "UserWarning: functional_call was passed multiple values for tied weights.
        # This behavior is deprecated and will be an error in future versions"
        # not 100% sure what this is, so far seems to be harmless. TODO investigate
        self.transformer.wte.weight = (
            self.lm_head.weight
        )  # https://paperswithcode.com/method/weight-tying

        # init all weights
        self.apply(self._init_weights)
        # apply special scaled init to the residual projections, per GPT-2 paper
        for pn, p in self.named_parameters():
            if pn.endswith("c_proj.weight"):
                torch.nn.init.normal_(
                    p, mean=0.0, std=0.02 / math.sqrt(2 * config.n_layer)
                )

        # report number of parameters
        print("number of parameters: %.2fM" % (self.get_num_params() / 1e6,))

    def get_num_params(self, non_embedding=True):
        """
        Return the number of parameters in the model.
        For non-embedding count (default), the position embeddings get subtracted.
        The token embeddings would too, except due to the parameter sharing these
        params are actually used as weights in the final layer, so we include them.
        """
        n_params = sum(p.numel() for p in self.parameters())
        if non_embedding:
            n_params -= self.transformer.wpe.weight.numel()
        return n_params

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        assert (
            t <= self.config.block_size
        ), f"Cannot forward sequence of length {t}, block size is only {self.config.block_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device)  # shape (t)

        # forward the GPT model itself
        tok_emb = self.transformer.wte(idx)  # token embeddings of shape (b, t, n_embd)
        pos_emb = self.transformer.wpe(pos)  # position embeddings of shape (t, n_embd)
        x = self.transformer.drop(tok_emb + pos_emb)
        for block in self.transformer.h:
#             print(f'processing {str(block)}')
            x = block(x)
        x = self.transformer.ln_f(x)

        if targets is not None:
            # if we are given some desired targets also calculate the loss
            logits = self.lm_head(x)
            loss = F.cross_entropy(
                logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1
            )
        else:
            # inference-time mini-optimization: only forward the lm_head on the very last position
            logits = self.lm_head(
                x[:, [-1], :]
            )  # note: using list [-1] to preserve the time dim
            loss = None

        return logits, loss
    

#     @torch.no_grad()
#     def generate(self, x_token_indices, maxNewTokens:int, topK=None, temperature = 1.0):       
#         '''
#         temperature: Temperature is a parameter used in natural language processing models to control the 
#                      degree of randomness or diversity in the generated tokens from an autoregressive language model.
#                      The logits are the unnormalized scores that the model assigns to each possible token before applying
#                      the softmax function to obtain the probabilities. The temperature parameter is used to scale the logits 
#                      by dividing them by a positive value. This affects the softmax function and changes the distribution of the probabilities.
#                      The impact of dividing the logits by temperature is as follows:
#                         * If the temperature is 1, then there is no scaling and the original logits are used. This results in 
#                         the most likely token being generated according to the model's confidence.
#                         * If the temperature is close to 0, then the scaling makes the logits very large and the softmax function 
#                         becomes very sharp. This results in the model always generating the token with the highest probability,
#                         which is equivalent to greedy decoding1.
#                         * If the temperature is higher than 1, then the scaling makes the logits smaller and the softmax function 
#                         becomes flatter. This results in the model generating tokens with lower probabilities more often, which 
#                         increases the diversity and randomness of the output1.
#                     Therefore, temperature can be used to trade-off between quality and diversity of the generated tokens from
#                     an autoregressive language model. A lower temperature leads to more coherent but less diverse outputs, 
#                     while a higher temperature leads to more diverse but less coherent outputs. 
#                     The optimal value of temperature may depend on the task and the preference of the user.
#         '''
#         #x_token_indices shape: (batch, sequence_length) 
#         for nextTokenIndex in range(maxNewTokens):
#             conditionedOn_x_token_indices_clipped = (x_token_indices 
#                                                      if x_token_indices.size(1) < self.config.block_size 
#                                                      else x_token_indices[:,-self.config.block_size:])


#             logits,_ = self(conditionedOn_x_token_indices_clipped) # run forward pass with the conditioned tokens

#             logits = logits[:,-1,:] #logits shape (batch,sequence,embedding) => take the last token's full embedding (:) for the batch (:)

#             #scale the logits by temperature
#             logits = logits / temperature

#             if topK is not None:
               
#                #take the top k embedding features from the logits and set the remaining to -infinity
#                #so that they are not connsidered during the softmax.
#                topKValues, _ =  torch.topk(logits,k=min(topK,logits.size(-1))) #(batch, sequence, topK) => picks topK embeddings for each sequence in the batch.

#                # Across batch (:), for all sequences (:), take the last embedding value (-1) i.e. the minimum value in the topK
# #                minValuesInTopKValues shape: (batch, sequence, 1)
#                minValuesInTopKValues = topKValues[:,:,[-1]] 

#                logits[logits < minValuesInTopKValues] = -float("Inf")
            
#             # apply softmax along the embedding features dimension (-1) to convert 
#             # logits (shape: batch, sequence, embedding/topK) to normalized probabilities 
#             probs = torch.nn.functional.softmax(logits,dim=-1)

#             next_token_index = torch.multinomial(probs, num_samples=1)

#             # append the sampled new token index to the sequence produced so far as an input for next iteration.
#             x_token_indices = torch.cat((x_token_indices, next_token_index), dim=1)
        
#         return x_token_indices

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        """
        Take a conditioning sequence of indices idx (LongTensor of shape (b,t)) and complete
        the sequence max_new_tokens times, feeding the predictions back into the model each time.
        Most likely you'll want to make sure to be in model.eval() mode of operation for this.
        """
        for _ in range(max_new_tokens):
            # if the sequence context is growing too long we must crop it at block_size
            idx_cond = (
                idx
                if idx.size(1) <= self.config.block_size
                else idx[:, -self.config.block_size :]
            )
            # forward the model to get the logits for the index in the sequence
            logits, _ = self(idx_cond)
            # pluck the logits at the final step and scale by desired temperature
            logits = logits[:, -1, :] / temperature
            # optionally crop the logits to only the top k options
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float("Inf")
            # apply softmax to convert logits to (normalized) probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to the running sequence and continue
            idx = torch.cat((idx, idx_next), dim=1)

        return idx

    @classmethod
    def from_pretrained(cls, model_type, override_args=None):
        assert model_type in {"gpt2", "gpt2-medium", "gpt2-large", "gpt2-xl"}
        override_args = override_args or {}  # default to empty dict
        # only dropout can be overridden see more notes below
        assert all(k == "dropout" for k in override_args)
        from transformers import GPT2LMHeadModel

        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            "gpt2": dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
            "gpt2-medium": dict(n_layer=24, n_head=16, n_embd=1024),  # 350M params
            "gpt2-large": dict(n_layer=36, n_head=20, n_embd=1280),  # 774M params
            "gpt2-xl": dict(n_layer=48, n_head=25, n_embd=1600),  # 1558M params
        }[model_type]
        print("forcing vocab_size=50257, block_size=1024, bias=True")
        config_args["vocab_size"] = 50257  # always 50257 for GPT model checkpoints
        config_args["block_size"] = 1024  # always 1024 for GPT model checkpoints
        config_args["bias"] = True  # always True for GPT model checkpoints
        # we can override the dropout rate, if desired
        if "dropout" in override_args:
            print(f"overriding dropout rate to {override_args['dropout']}")
            config_args["dropout"] = override_args["dropout"]
        # create a from-scratch initialized minGPT model
        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [
            k for k in sd_keys if not k.endswith(".attn.bias")
        ]  # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [
            k for k in sd_keys_hf if not k.endswith(".attn.masked_bias")
        ]  # ignore these, just a buffer
        sd_keys_hf = [
            k for k in sd_keys_hf if not k.endswith(".attn.bias")
        ]  # same, just the mask (buffer)
        transposed = [
            "attn.c_attn.weight",
            "attn.c_proj.weight",
            "mlp.c_fc.weight",
            "mlp.c_proj.weight",
        ]
        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(
            sd_keys
        ), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model


    def configure_optimizers(self, weight_decay, learning_rate, betas, device_type):
        # start with all of the candidate parameters
        param_dict = {pn: p for pn, p in self.named_parameters()}
        # filter out those that do not require grad
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
        # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {"params": decay_params, "weight_decay": weight_decay},
            {"params": nodecay_params, "weight_decay": 0.0},
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        print(
            f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters"
        )
        print(
            f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters"
        )
        # Create AdamW optimizer and use the fused version if it is available
        fused_available = "fused" in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == "cuda"
        extra_args = dict(fused=True) if use_fused else dict()
        optimizer = torch.optim.AdamW(
            optim_groups, lr=learning_rate, betas=betas, **extra_args
        )
        print(f"using fused AdamW: {use_fused}")

        return optimizer
    
    def crop_block_size(self, block_size):
        # model surgery to decrease the block size if necessary
        # e.g. we may load the GPT2 pretrained model checkpoint (block size 1024)
        # but want to use a smaller block size for some smaller, simpler model
        assert block_size <= self.config.block_size
        self.config.block_size = block_size
        self.transformer.wpe.weight = nn.Parameter(
            self.transformer.wpe.weight[:block_size]
        )
        for block in self.transformer.h:
            if hasattr(block.attn, "bias"):
                block.attn.bias = block.attn.bias[:, :, :block_size, :block_size]    

In [ ]:
gptConfig = GPTConfig()
customGptModel = GPT(gptConfig)

bl = Block(gptConfig)

In [ ]:
import torch
from transformers import AutoTokenizer, GPT2LMHeadModel

tokenizer = AutoTokenizer.from_pretrained("gpt2")
hf_gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = hf_gpt2_model(**inputs, labels=inputs["input_ids"])
loss = outputs.loss
logits = outputs.logits

print(inputs["input_ids"].shape)
print(logits.shape)

In [ ]:
import torchinfo

torchinfo.summary(model=hf_gpt2_model,depth=6)

In [ ]:
from prettytable import PrettyTable
def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

count_parameters(hf_gpt2_model)

In [ ]:
count_parameters(customGptModel)
# torchinfo.summary(customGptModel,depth=6)

Now, let's prepare data for fine-turning the pre-trained GPT2 model.

In [ ]:
import fitz

pdf_fileName = './data/InvestmentBanking.pdf'
pdf_fileName = './data/ModernBanking.pdf'
out_fileName = pdf_fileName.replace('.pdf','.txt')
print(out_fileName)
doc = fitz.open(pdf_fileName) # open a document
out = open(out_fileName, "wb") # create a text output
for page in doc: # iterate the document pages
	text = page.get_text().encode("utf8") # get plain text (is in UTF-8)
	out.write(text) # write text of page
	out.write(bytes((12,))) # write page delimiter (form feed 0x0C)
out.close()

Let's pre-process the data to make it compatible with the model inputs.

In [ ]:
import os
import requests
import tiktoken
import numpy as np

input_file_path = './data/ModernBanking.txt'
out_folder_path = './data/'
with open(input_file_path, 'r') as f:
    data = f.read()
n = len(data)
print(f'Total length of file: {input_file_path} is: {n}')

train_data = data[:int(n*0.9)]
val_data = data[int(n*0.9):]

# encode with tiktoken gpt2 bpe
enc = tiktoken.get_encoding("gpt2")
train_ids = enc.encode_ordinary(train_data)
val_ids = enc.encode_ordinary(val_data)
print(f"train has {len(train_ids):,} tokens")
print(f"val has {len(val_ids):,} tokens")

# export to bin files
train_ids = np.array(train_ids, dtype=np.uint16)
val_ids = np.array(val_ids, dtype=np.uint16)
train_ids.tofile(os.path.join(out_folder_path, 'train.bin'))
val_ids.tofile(os.path.join(out_folder_path, 'val.bin'))

In [ ]:
##smaller model config

# -----------------------------------------------------------------------------
# default config values designed to train a gpt2 (124M) on OpenWebText
# I/O
out_dir = "out"
eval_interval = 5 #2000
log_interval = 1
eval_iters = 20 #200
eval_only = False  # if True, script exits right after the first eval
always_save_checkpoint = True  # if True, always save a checkpoint after each eval
init_from = "gpt2"  # 'scratch' or 'resume' or 'gpt2*'
# wandb logging
wandb_log = False  # disabled by default
wandb_project = "owt"
wandb_run_name = "gpt2"  # 'run' + str(time.time())
# data
dataset = "modernBanking"
gradient_accumulation_steps = 5 * 8  # used to simulate larger batch sizes
batch_size = 6  # if gradient_accumulation_steps > 1, this is the micro-batch size
block_size = 64 #1024
# model
n_layer = 6 #12
n_head = 6 #12
n_embd = 128
dropout = 0.0  # for pretraining 0 is good, for finetuning try 0.1+
bias = False  # do we use bias inside LayerNorm and Linear layers?
# adamw optimizer
learning_rate = 6e-4  # max learning rate
max_iters = 2000 #600000  # total number of training iterations
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0  # clip gradients at this value, or disable if == 0.0
# learning rate decay settings
decay_lr = True  # whether to decay the learning rate
warmup_iters = 200 #2000  # how many steps to warm up for
lr_decay_iters = 2000 #600000  # should be ~= max_iters per Chinchilla
min_lr = 6e-5  # minimum learning rate, should be ~= learning_rate/10 per Chinchilla
# DDP settings
backend = "nccl"  # 'nccl', 'gloo', etc.
# system
device = (
    "cuda:0"  # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1' etc., or try 'mps' on macbooks
)
dtype = (
    "bfloat16"
    if torch.cuda.is_available() and torch.cuda.is_bf16_supported()
    else "float16"
)  # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
compile = True  # use PyTorch 2.0 to compile the model to be faster

isWindowsOS = True


config_keys = [
    k
    for k, v in globals().items()
    if not k.startswith("_") and isinstance(v, (int, float, bool, str))
]
# exec(open("configurator.py").read())  # overrides from command line or config file
config = {k: globals()[k] for k in config_keys}  # will be useful for logging

print(str(config))

In [4]:
# -----------------------------------------------------------------------------
# default config values designed to train a gpt2 (124M) on OpenWebText
# I/O
out_dir = "out"
eval_interval = 50
log_interval = 1
eval_iters = 200
eval_only = False  # if True, script exits right after the first eval
always_save_checkpoint = True  # if True, always save a checkpoint after each eval
init_from = "gpt2"  # 'scratch' or 'resume' or 'gpt2*'
# wandb logging
wandb_log = False  # disabled by default
wandb_project = "owt"
wandb_run_name = "gpt2"  # 'run' + str(time.time())
# data
dataset = "modernBanking"
gradient_accumulation_steps = 5 * 8  # used to simulate larger batch sizes
batch_size = 4  # if gradient_accumulation_steps > 1, this is the micro-batch size
block_size = 256
# model
n_layer = 10
n_head = 10
n_embd = 256
dropout = 0.1  # for pretraining 0 is good, for finetuning try 0.1+
bias = False  # do we use bias inside LayerNorm and Linear layers?
# adamw optimizer
learning_rate = 6e-4  # max learning rate
max_iters = 600000  # total number of training iterations
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0  # clip gradients at this value, or disable if == 0.0
# learning rate decay settings
decay_lr = True  # whether to decay the learning rate
warmup_iters = 2000  # how many steps to warm up for
lr_decay_iters = 600000  # should be ~= max_iters per Chinchilla
min_lr = 6e-5  # minimum learning rate, should be ~= learning_rate/10 per Chinchilla
# DDP settings
backend = "nccl"  # 'nccl', 'gloo', etc.
# system
device = (
    "cuda"  # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1' etc., or try 'mps' on macbooks
)
dtype = (
    "bfloat16"
    if torch.cuda.is_available() and torch.cuda.is_bf16_supported()
    else "float16"
)  # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
compile = True  # use PyTorch 2.0 to compile the model to be faster

isWindowsOS = True

# -----------------------------------------------------------------------------
config_keys = [
    k
    for k, v in globals().items()
    if not k.startswith("_") and isinstance(v, (int, float, bool, str))
]

config = {k: globals()[k] for k in config_keys}  # will be useful for logging

In [5]:
import torch
from contextlib import nullcontext
import os
import numpy as np

tokens_per_iter = gradient_accumulation_steps * batch_size * block_size
print(f"tokens per iteration will be: {tokens_per_iter:,}")

torch.manual_seed(1337)
torch.backends.cuda.matmul.allow_tf32 = True  # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True  # allow tf32 on cudnn
device_type = "cuda" if "cuda" in device else "cpu"  # for later use in torch.autocast

print(f'device_type: {device_type}')

print(f'dtype: {dtype}')
# note: float16 data type will automatically use a GradScaler
ptdtype = {
    "float32": torch.float32,
    "bfloat16": torch.bfloat16,
    "float16": torch.float16,
}[dtype]

print(f'ptdtype: {ptdtype}')

ctx = (
    nullcontext()
    if device_type == "cpu"
    else torch.amp.autocast(device_type=device_type, dtype=ptdtype)
)

print(f'ctx: {ctx}')

# poor man's data loader
data_dir = os.path.join("./data", dataset)
train_data = np.memmap(os.path.join(data_dir, "train.bin"), dtype=np.uint16, mode="r")
val_data = np.memmap(os.path.join(data_dir, "val.bin"), dtype=np.uint16, mode="r")

print(f'train data length: {len(train_data)}')
print(f'val data length: {len(val_data)}')


tokens per iteration will be: 40,960
device_type: cuda
dtype: float16
ptdtype: torch.float16
ctx: <torch.amp.autocast_mode.autocast object at 0x0000020896851A20>
train data length: 506290
val data length: 79438


In [6]:
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack(
        [torch.from_numpy((data[i : i + block_size]).astype(np.int64)) for i in ix]
    )
    y = torch.stack(
        [
            torch.from_numpy((data[i + 1 : i + 1 + block_size]).astype(np.int64))
            for i in ix
        ]
    )
    if device_type == "cuda":
        # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(
            device, non_blocking=True
        )
    else:
        x, y = x.to(device), y.to(device)
    return x, y

In [7]:
# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))  # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)

In [8]:
# helps estimate an arbitrarily accurate loss over either split using many batches
@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            with ctx:
                logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [9]:
from dataclasses import dataclass

@dataclass
class GPTConfig:
    block_size: int = 1024
    vocab_size: int = 50304  # GPT-2 vocab_size of 50257, padded up to nearest multiple of 64 for efficiency
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768
    dropout: float = 0.0
    bias: bool = True  # True: bias in Linears and LayerNorms, like GPT-2. False: a bit better and faster

In [10]:
iter_num = 0
best_val_loss = 1e9

# model init
model_args = dict(
    n_layer=n_layer,
    n_head=n_head,
    n_embd=n_embd,
    block_size=block_size,
    bias=bias,
    vocab_size=None,
    dropout=dropout,
)

print(f"Initializing from OpenAI GPT-2 weights: {init_from}")
# initialize from OpenAI GPT-2 weights
override_args = dict(dropout=dropout)
model = GPT.from_pretrained(init_from, override_args)
# read off the created config params, so we can store them into checkpoint correctly
for k in ["n_layer", "n_head", "n_embd", "block_size", "bias", "vocab_size"]:
        model_args[k] = getattr(model.config, k)
        
# crop down the model block size if desired, using model surgery
if block_size < model.config.block_size:
    model.crop_block_size(block_size)
    model_args[
        "block_size"
    ] = block_size  # so that the checkpoint will have the right value

model.to(device)

# initialize a GradScaler. If enabled=False scaler is a no-op
scaler = torch.cuda.amp.GradScaler(enabled=(dtype == "float16"))

print(f'Gradscaler enabled: {scaler.is_enabled}')

# optimizer
optimizer = model.configure_optimizers(
    weight_decay, learning_rate, (beta1, beta2), device_type
)

if compile and not isWindowsOS:
    print("compiling the model... (takes a ~minute)")
    unoptimized_model = model
    model = torch.compile(model)  # requires PyTorch 2.0

# logging
if wandb_log:
    import wandb
    
    wandb.init(project=wandb_project, name=wandb_run_name, config=config)

Initializing from OpenAI GPT-2 weights: gpt2
loading weights from pretrained gpt: gpt2
forcing vocab_size=50257, block_size=1024, bias=True
overriding dropout rate to 0.1
number of parameters: 123.65M
Gradscaler enabled: <bound method GradScaler.is_enabled of <torch.cuda.amp.grad_scaler.GradScaler object at 0x0000020896853D60>>
num decayed parameter tensors: 50, with 123,728,640 parameters
num non-decayed parameter tensors: 98, with 121,344 parameters
using fused AdamW: True


In [11]:
# training loop
import time


X, Y = get_batch("train")  # fetch the very first batch
t0 = time.time()
local_iter_num = 0  # number of iterations in the lifetime of this process
raw_model = model
running_mfu = -1.0
while True:
    # determine and set the learning rate for this iteration
    lr = get_lr(iter_num) if decay_lr else learning_rate
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

    # evaluate the loss on train/val sets and write checkpoints
    if iter_num % eval_interval == 0:
        losses = estimate_loss(model)
        print(
            f"step {iter_num}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}"
        )
        if wandb_log:
            wandb.log(
                {
                    "iter": iter_num,
                    "train/loss": losses["train"],
                    "val/loss": losses["val"],
                    "lr": lr,
                    "mfu": running_mfu * 100,  # convert to percentage
                }
            )
        if losses["val"] < best_val_loss or always_save_checkpoint:
            best_val_loss = losses["val"]
            if iter_num > 0:
                checkpoint = {
                    "model": raw_model.state_dict(),
                    "optimizer": optimizer.state_dict(),
                    "model_args": model_args,
                    "iter_num": iter_num,
                    "best_val_loss": best_val_loss,
                    "config": config,
                }
                print(f"saving checkpoint to {out_dir}")
                torch.save(checkpoint, os.path.join(out_dir, "ckpt.pt"))
    if iter_num == 0 and eval_only:
        break

    # forward backward update, with optional gradient accumulation to simulate larger batch size
    # and using the GradScaler if data type is float16
    for micro_step in range(gradient_accumulation_steps):
        # if ddp:
        #     # in DDP training we only need to sync gradients at the last micro step.
        #     # the official way to do this is with model.no_sync() context manager, but
        #     # I really dislike that this bloats the code and forces us to repeat code
        #     # looking at the source of that context manager, it just toggles this variable
        #     model.require_backward_grad_sync = (
        #         micro_step == gradient_accumulation_steps - 1
        #     )
        with ctx:
            logits, loss = model(X, Y)
            loss = (
                loss / gradient_accumulation_steps
            )  # scale the loss to account for gradient accumulation
        
#         print(f'loss in micro step: {micro_step} out of {gradient_accumulation_steps} steps is: {loss}')

        # immediately async prefetch next batch while model is doing the forward pass on the GPU
        X, Y = get_batch("train")
        # backward pass, with gradient scaling if training in fp16
        scaler.scale(loss).backward()
    # clip the gradient
    if grad_clip != 0.0:
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
    # step the optimizer and scaler if training in fp16
    scaler.step(optimizer)
    scaler.update()
    # flush the gradients as soon as we can, no need for this memory anymore
    optimizer.zero_grad(set_to_none=True)

    # timing and logging
    t1 = time.time()
    dt = t1 - t0
    t0 = t1
    if iter_num % log_interval == 0:
        # get loss as float. note: this is a CPU-GPU sync point
        # scale up to undo the division above, approximating the true total loss (exact would have been a sum)
        lossf = loss.item() * gradient_accumulation_steps
        # if local_iter_num >= 5:  # let the training loop settle a bit
        #     mfu = raw_model.estimate_mfu(batch_size * gradient_accumulation_steps, dt)
        #     running_mfu = mfu if running_mfu == -1.0 else 0.9 * running_mfu + 0.1 * mfu
        print(
            f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms, mfu {running_mfu*100:.2f}%"
        )
    iter_num += 1
    local_iter_num += 1

    # termination conditions
    if iter_num > max_iters:
        break

step 0: train loss 3.8691, val loss 3.4823
iter 0: loss 4.5197, time 152703.76ms, mfu -100.00%
iter 1: loss 4.2347, time 40571.80ms, mfu -100.00%
iter 2: loss 4.0972, time 40575.38ms, mfu -100.00%
iter 3: loss 4.1663, time 40589.92ms, mfu -100.00%
iter 4: loss 4.0585, time 40588.90ms, mfu -100.00%
iter 5: loss 3.5662, time 40582.92ms, mfu -100.00%
iter 6: loss 4.4242, time 40645.90ms, mfu -100.00%
iter 7: loss 4.3115, time 40596.76ms, mfu -100.00%
iter 8: loss 4.1044, time 40648.87ms, mfu -100.00%
iter 9: loss 4.0423, time 40629.83ms, mfu -100.00%
iter 10: loss 4.1185, time 40595.06ms, mfu -100.00%
iter 11: loss 4.4053, time 40666.93ms, mfu -100.00%
iter 12: loss 4.5851, time 40696.30ms, mfu -100.00%
iter 13: loss 4.0787, time 40649.34ms, mfu -100.00%
iter 14: loss 4.1111, time 40651.41ms, mfu -100.00%
iter 15: loss 3.8644, time 40683.68ms, mfu -100.00%
iter 16: loss 4.3184, time 40662.23ms, mfu -100.00%
iter 17: loss 4.0489, time 40642.41ms, mfu -100.00%
iter 18: loss 4.0707, time 406

iter 152: loss 2.7737, time 40711.25ms, mfu -100.00%
iter 153: loss 3.1077, time 40830.17ms, mfu -100.00%
iter 154: loss 2.7244, time 40738.40ms, mfu -100.00%
iter 155: loss 2.6497, time 40744.90ms, mfu -100.00%
iter 156: loss 2.9749, time 40768.09ms, mfu -100.00%
iter 157: loss 2.7626, time 40767.79ms, mfu -100.00%
iter 158: loss 3.0569, time 40753.89ms, mfu -100.00%
iter 159: loss 2.8146, time 40776.87ms, mfu -100.00%
iter 160: loss 2.9872, time 40756.46ms, mfu -100.00%
iter 161: loss 2.7289, time 40754.24ms, mfu -100.00%
iter 162: loss 2.6785, time 40765.74ms, mfu -100.00%
iter 163: loss 2.8144, time 40843.45ms, mfu -100.00%
iter 164: loss 2.5314, time 40710.49ms, mfu -100.00%
iter 165: loss 2.8418, time 40734.80ms, mfu -100.00%
iter 166: loss 2.7512, time 40781.18ms, mfu -100.00%
iter 167: loss 3.1973, time 40743.57ms, mfu -100.00%
iter 168: loss 2.8734, time 40777.81ms, mfu -100.00%
iter 169: loss 3.0638, time 40805.22ms, mfu -100.00%
iter 170: loss 2.8456, time 40831.78ms, mfu -1

iter 303: loss 2.2213, time 40725.05ms, mfu -100.00%
iter 304: loss 2.4158, time 40713.91ms, mfu -100.00%
iter 305: loss 2.4905, time 40800.81ms, mfu -100.00%
iter 306: loss 2.5885, time 40854.00ms, mfu -100.00%
iter 307: loss 2.4223, time 40717.55ms, mfu -100.00%
iter 308: loss 2.4857, time 40760.38ms, mfu -100.00%
iter 309: loss 2.6738, time 40689.67ms, mfu -100.00%
iter 310: loss 2.4517, time 40734.31ms, mfu -100.00%
iter 311: loss 2.5622, time 40797.37ms, mfu -100.00%
iter 312: loss 2.7362, time 40799.16ms, mfu -100.00%
iter 313: loss 2.8516, time 40765.53ms, mfu -100.00%
iter 314: loss 2.5169, time 40811.18ms, mfu -100.00%
iter 315: loss 2.2502, time 40693.85ms, mfu -100.00%
iter 316: loss 2.2824, time 40728.14ms, mfu -100.00%
iter 317: loss 2.4637, time 40688.23ms, mfu -100.00%
iter 318: loss 2.4263, time 40659.56ms, mfu -100.00%
iter 319: loss 2.5210, time 40633.00ms, mfu -100.00%
iter 320: loss 2.4014, time 40670.79ms, mfu -100.00%
iter 321: loss 2.5378, time 40670.00ms, mfu -1

iter 454: loss 2.1375, time 40677.10ms, mfu -100.00%
iter 455: loss 2.2053, time 40699.10ms, mfu -100.00%
iter 456: loss 2.4016, time 40724.15ms, mfu -100.00%
iter 457: loss 2.0805, time 40739.62ms, mfu -100.00%
iter 458: loss 1.8967, time 40708.60ms, mfu -100.00%
iter 459: loss 2.1392, time 40806.30ms, mfu -100.00%
iter 460: loss 2.1038, time 40762.41ms, mfu -100.00%
iter 461: loss 2.1134, time 40741.31ms, mfu -100.00%
iter 462: loss 1.9972, time 40754.82ms, mfu -100.00%
iter 463: loss 2.1489, time 40745.40ms, mfu -100.00%
iter 464: loss 2.1769, time 40701.38ms, mfu -100.00%
iter 465: loss 2.0766, time 40742.48ms, mfu -100.00%
iter 466: loss 1.8356, time 40758.79ms, mfu -100.00%
iter 467: loss 2.1532, time 40733.38ms, mfu -100.00%
iter 468: loss 2.0430, time 40735.96ms, mfu -100.00%
iter 469: loss 2.2716, time 40729.93ms, mfu -100.00%
iter 470: loss 1.9657, time 40684.16ms, mfu -100.00%
iter 471: loss 1.9041, time 40720.71ms, mfu -100.00%
iter 472: loss 1.9053, time 40744.12ms, mfu -1

iter 605: loss 1.6134, time 40693.34ms, mfu -100.00%
iter 606: loss 1.6291, time 40847.61ms, mfu -100.00%
iter 607: loss 1.8166, time 40648.62ms, mfu -100.00%
iter 608: loss 1.6683, time 40597.22ms, mfu -100.00%
iter 609: loss 1.9093, time 40608.99ms, mfu -100.00%
iter 610: loss 1.7614, time 40664.47ms, mfu -100.00%
iter 611: loss 1.6427, time 40643.25ms, mfu -100.00%
iter 612: loss 1.6602, time 40667.60ms, mfu -100.00%
iter 613: loss 1.6709, time 40666.00ms, mfu -100.00%
iter 614: loss 1.1384, time 40656.00ms, mfu -100.00%
iter 615: loss 1.6192, time 40703.49ms, mfu -100.00%
iter 616: loss 1.6002, time 40701.05ms, mfu -100.00%
iter 617: loss 1.7900, time 40687.95ms, mfu -100.00%
iter 618: loss 1.5131, time 40698.19ms, mfu -100.00%
iter 619: loss 1.5105, time 40808.46ms, mfu -100.00%
iter 620: loss 1.9595, time 40870.28ms, mfu -100.00%
iter 621: loss 1.5282, time 40780.80ms, mfu -100.00%
iter 622: loss 1.5858, time 40845.50ms, mfu -100.00%
iter 623: loss 1.0325, time 40769.98ms, mfu -1

iter 756: loss 1.1097, time 40722.64ms, mfu -100.00%
iter 757: loss 1.3419, time 40737.43ms, mfu -100.00%
iter 758: loss 1.1092, time 40617.72ms, mfu -100.00%
iter 759: loss 1.1439, time 40714.46ms, mfu -100.00%
iter 760: loss 0.9590, time 40961.19ms, mfu -100.00%
iter 761: loss 1.2048, time 40189.04ms, mfu -100.00%
iter 762: loss 1.2707, time 40227.38ms, mfu -100.00%
iter 763: loss 1.2193, time 40200.49ms, mfu -100.00%
iter 764: loss 1.1586, time 40207.99ms, mfu -100.00%
iter 765: loss 1.0979, time 40211.75ms, mfu -100.00%
iter 766: loss 1.1406, time 40191.49ms, mfu -100.00%
iter 767: loss 1.1492, time 40246.38ms, mfu -100.00%
iter 768: loss 1.2516, time 40206.85ms, mfu -100.00%
iter 769: loss 1.2451, time 40208.13ms, mfu -100.00%
iter 770: loss 1.1286, time 40238.91ms, mfu -100.00%
iter 771: loss 0.9660, time 40193.66ms, mfu -100.00%
iter 772: loss 1.0393, time 40236.99ms, mfu -100.00%
iter 773: loss 1.1541, time 40222.38ms, mfu -100.00%
iter 774: loss 1.3073, time 40211.04ms, mfu -1

iter 907: loss 0.7447, time 40587.83ms, mfu -100.00%
iter 908: loss 0.6913, time 40513.85ms, mfu -100.00%
iter 909: loss 0.7632, time 40524.33ms, mfu -100.00%
iter 910: loss 0.7206, time 40554.26ms, mfu -100.00%
iter 911: loss 0.6564, time 40538.95ms, mfu -100.00%
iter 912: loss 0.6593, time 40581.05ms, mfu -100.00%
iter 913: loss 0.8075, time 40585.99ms, mfu -100.00%
iter 914: loss 0.7789, time 40553.48ms, mfu -100.00%
iter 915: loss 0.7014, time 40529.48ms, mfu -100.00%
iter 916: loss 0.7738, time 40539.69ms, mfu -100.00%
iter 917: loss 0.7005, time 40542.01ms, mfu -100.00%
iter 918: loss 0.8415, time 40602.40ms, mfu -100.00%
iter 919: loss 0.7466, time 40575.99ms, mfu -100.00%
iter 920: loss 0.6994, time 40566.79ms, mfu -100.00%
iter 921: loss 0.7404, time 40538.26ms, mfu -100.00%
iter 922: loss 0.7251, time 40550.86ms, mfu -100.00%
iter 923: loss 0.8241, time 40526.06ms, mfu -100.00%
iter 924: loss 0.8315, time 40523.99ms, mfu -100.00%
iter 925: loss 0.9703, time 40641.30ms, mfu -1

iter 1057: loss 0.4750, time 40572.27ms, mfu -100.00%
iter 1058: loss 0.6179, time 40566.03ms, mfu -100.00%
iter 1059: loss 0.4672, time 40617.79ms, mfu -100.00%
iter 1060: loss 0.5321, time 40550.26ms, mfu -100.00%
iter 1061: loss 0.4838, time 40548.61ms, mfu -100.00%
iter 1062: loss 0.3737, time 40571.90ms, mfu -100.00%
iter 1063: loss 0.7142, time 40578.33ms, mfu -100.00%
iter 1064: loss 0.4783, time 40546.52ms, mfu -100.00%
iter 1065: loss 0.4598, time 40554.89ms, mfu -100.00%
iter 1066: loss 0.4807, time 40590.07ms, mfu -100.00%
iter 1067: loss 0.3923, time 40487.56ms, mfu -100.00%
iter 1068: loss 0.4836, time 40557.87ms, mfu -100.00%
iter 1069: loss 0.5034, time 40556.86ms, mfu -100.00%
iter 1070: loss 0.3971, time 40531.73ms, mfu -100.00%
iter 1071: loss 0.3868, time 40551.91ms, mfu -100.00%
iter 1072: loss 0.4608, time 40570.76ms, mfu -100.00%
iter 1073: loss 0.4634, time 40535.76ms, mfu -100.00%
iter 1074: loss 0.5456, time 40586.56ms, mfu -100.00%
iter 1075: loss 0.4890, time

KeyboardInterrupt: 

In [12]:
out_dir = 'out' # ignored if init_from is not 'resume'
start = "\n" # or "<|endoftext|>" or etc. Can also specify a file, use as: "FILE:prompt.txt"
start = "A Derivative is a contract"
start = "Non-banks are"
start = "Risk is defined"
num_samples = 3 # number of samples to draw
max_new_tokens = 500 # number of tokens generated in each sample
temperature = 0.8 # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
top_k = 200 # retain only the top_k most likely tokens, clamp others to have 0 probability
seed = 1337
device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1', etc.
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32' or 'bfloat16' or 'float16'
compile = False # use PyTorch 2.0 to compile the model to be faster

In [13]:
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

In [14]:
import os

ckpt_path = os.path.join(out_dir, 'ckpt.pt')
checkpoint = torch.load(ckpt_path, map_location=device)
gptconf = GPTConfig(**checkpoint['model_args'])
model = GPT(gptconf)
state_dict = checkpoint['model']
unwanted_prefix = '_orig_mod.'
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
model.load_state_dict(state_dict)

model.eval()
model.to(device)
if compile:
    model = torch.compile(model) # requires PyTorch 2.0 (optional)

number of parameters: 123.65M


OutOfMemoryError: CUDA out of memory. Tried to allocate 148.00 MiB (GPU 0; 4.00 GiB total capacity; 3.36 GiB already allocated; 0 bytes free; 3.49 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
import tiktoken

enc = tiktoken.get_encoding("gpt2")
encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
decode = lambda l: enc.decode(l)

start_ids = encode(start)
x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])

In [ ]:
# run generation
with torch.no_grad():
    with ctx:
        for k in range(num_samples):
            y = model.generate(x, max_new_tokens, temperature=temperature, top_k=top_k)
            print(decode(y[0].tolist()))
            print('---------------')

In [ ]:
# print(str(globals().items()))